In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

In [2]:
from project_heart.enums import *

In [3]:
dir = Path("C:/Users/igorp/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
# dir = Path("C:/Users/igornobrega/USF_Research/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models")
files = os.listdir(dir)

file_id = 7

print("File:", files[file_id])

lv = LV.from_pyvista_read(dir/files[file_id], identifier="elemTag", threshold=[0, 1])

File: Full_Heart_Mesh_16.vtk


In [4]:
import tetgen
lv_surf = lv.get_surface_mesh().triangulate().smooth(350)
tet = tetgen.TetGen(lv_surf)
tet.make_manifold()
tet.tetrahedralize(order=1, 
                   nobisect=True,
                  #  quality=False,
                   mindihedral=100, 
                   minratio=1.1,
                  #  steinerleft=-1
                   )
lv_tet = LV().from_pyvista_dataset(tet.grid)
lv_tet

In [5]:
lv_tet.plot()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [6]:
lv_tet.guess_epi_endo_based_on_surf_normals(90.0)
lv_tet.plot("surface", scalars=LV_MESH_DATA.EPI_ENDO_GUESS.value)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [7]:
lv_tet.identify_base_and_apex_regions()

(array([6., 6., 6., ..., 0., 6., 0.]), array([6., 6., 6., ..., 0., 0., 0.]))

In [8]:
lv_tet.identify_mitral_and_aortic_regions(
                                      a1=0.4,  # coeff for radial distance computation
                                      a2 = 0.5,
                                      a3=0.4,
                                      a4=80,
                                      a5=120,
                                      
                                      m1=0.18,
                                      m2=0.005,
                                      m3=0.05,
                                      )
lv_tet.plot("mesh", scalars=LV_MESH_DATA.AM_CLUSTERS.value, cmap="tab20")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [12]:
lv_tet.identify_surfaces(
  endo_epi_args=dict(threshold=90.0),
  apex_base_args=dict(ab_ql=0.05, ab_qh=0.70),
  aortic_mitral_args=dict(a1=0.4,
                          a2 = 0.5,
                          a3=0.4,
                          a4=80,
                          a5=120,
                          m1=0.18,
                          m2=0.005,
                          m3=0.05)
)
lv_tet.plot("surface", scalars="TESTE", cmap="tab20", show_edges=True)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…